In [1]:
import sqlite3

conn = sqlite3.connect("triage.db")
cursor = conn.cursor()

cursor.execute("PRAGMA foreign_keys = ON")

cursor.execute("""
CREATE TABLE IF NOT EXISTS doctors (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  specialty TEXT NOT NULL,
  department TEXT NOT NULL,
  location TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS labs (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  location TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS availability (
  id INTEGER PRIMARY KEY,
  resource_type TEXT NOT NULL CHECK (resource_type IN ('doctor','lab')),
  resource_id INTEGER NOT NULL,
  slot_start TEXT NOT NULL,
  slot_end TEXT NOT NULL,
  is_available INTEGER NOT NULL CHECK (is_available IN (0,1)),
  UNIQUE (resource_type, resource_id, slot_start)
);
""")

cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_avail_lookup
ON availability (resource_type, resource_id, is_available, slot_start);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS appointments (
  id INTEGER PRIMARY KEY,
  user_id TEXT NOT NULL,
  kind TEXT NOT NULL,
  resource_id INTEGER,
  resource_type TEXT,
  requested_slot TEXT,
  booked_slot TEXT,
  status TEXT NOT NULL,
  suggested_alternatives TEXT,
  created_at TEXT NOT NULL DEFAULT (datetime('now'))
);
""")

conn.commit()
conn.close()
print("Schema initialized")

Schema initialized


In [2]:
import sqlite3

conn = sqlite3.connect("triage.db")
cursor = conn.cursor()

# Insert 5 doctors
doctors = [
    ("Dr. Arjun Mehta", "Cardiology", "Heart Care", "Building A"),
    ("Dr. Priya Sharma", "Neurology", "Brain & Spine", "Building B"),
    ("Dr. Rakesh Nair", "Orthopedics", "Bone & Joint", "Building C"),
    ("Dr. Sneha Kapoor", "Dermatology", "Skin Care", "Building D"),
    ("Dr. Vivek Rao", "Pediatrics", "Child Care", "Building E"),
]
cursor.executemany("INSERT INTO doctors (name, specialty, department, location) VALUES (?, ?, ?, ?)", doctors)

# Insert 5 labs
labs = [
    ("Radiology Lab", "Building F"),
    ("Pathology Lab", "Building G"),
    ("Biochemistry Lab", "Building H"),
    ("Microbiology Lab", "Building I"),
    ("Hematology Lab", "Building J"),
]
cursor.executemany("INSERT INTO labs (name, location) VALUES (?, ?)", labs)

# Insert 5 availability slots (doctor type)
availability = [
    ("doctor", 1, "2025-12-01 09:00", "2025-12-01 09:30", 1),
    ("doctor", 2, "2025-12-01 10:00", "2025-12-01 10:30", 1),
    ("doctor", 3, "2025-12-01 11:00", "2025-12-01 11:30", 1),
    ("doctor", 4, "2025-12-01 12:00", "2025-12-01 12:30", 1),
    ("doctor", 5, "2025-12-01 14:00", "2025-12-01 14:30", 1),
]
cursor.executemany("""
INSERT INTO availability (resource_type, resource_id, slot_start, slot_end, is_available)
VALUES (?, ?, ?, ?, ?)
""", availability)

# Insert 5 appointments
appointments = [
    ("user001", "doctor", 1, "doctor", "2025-12-01 09:00", "2025-12-01 09:00", "booked", "[]"),
    ("user002", "doctor", 2, "doctor", "2025-12-01 10:00", "2025-12-01 10:00", "booked", "[]"),
    ("user003", "doctor", 3, "doctor", "2025-12-01 11:00", "2025-12-01 11:00", "pending", "[]"),
    ("user004", "lab", 1, "lab", "2025-12-02 09:00", "2025-12-02 09:00", "booked", "[]"),
    ("user005", "lab", 2, "lab", "2025-12-02 10:00", "2025-12-02 10:00", "cancelled", "[]"),
]
cursor.executemany("""
INSERT INTO appointments (user_id, kind, resource_id, resource_type, requested_slot, booked_slot, status, suggested_alternatives)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", appointments)

conn.commit()
conn.close()

print("Inserted 5 records each into doctors, labs, availability, and appointments tables.")


Inserted 5 records each into doctors, labs, availability, and appointments tables.


In [3]:
# import sqlite3    
# doctor_name = "Dr. Arjun Mehta"
# with sqlite3.connect("triage.db") as conn:
#     c = conn.cursor()
#     c.execute("SELECT id FROM doctors WHERE name=?", (doctor_name,))
#     row = c.fetchone()
#     if not row:
#         print("No Record")
#     # doctor_id = row[0]
# print(doctor_id)

import sqlite3
conn = sqlite3.connect("triage.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM doctors")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

(1, 'Dr. Arjun Mehta', 'Cardiology', 'Heart Care', 'Building A')
(2, 'Dr. Priya Sharma', 'Neurology', 'Brain & Spine', 'Building B')
(3, 'Dr. Rakesh Nair', 'Orthopedics', 'Bone & Joint', 'Building C')
(4, 'Dr. Sneha Kapoor', 'Dermatology', 'Skin Care', 'Building D')
(5, 'Dr. Vivek Rao', 'Pediatrics', 'Child Care', 'Building E')


In [5]:
import requests
import xml.etree.ElementTree as ET

# MedlinePlus Health Topics API URL
url = "https://wsearch.nlm.nih.gov/ws/query"
params = {
    "db": "healthTopics",
    "term": "asthma"
}

# Make the request
response = requests.get(url, params=params)

if response.status_code == 200:
    # Parse XML response
    root = ET.fromstring(response.text)

    # Extract useful info
    results = []
    for document in root.findall(".//document"):
        title = document.find("content[@name='title']").text if document.find("content[@name='title']") is not None else None
        url = document.find("content[@name='url']").text if document.find("content[@name='url']") is not None else None
        summary = document.find("content[@name='FullSummary']").text if document.find("content[@name='FullSummary']") is not None else None

        results.append({
            "title": title,
            "url": url,
            "summary": summary
        })

    # Print results
    for r in results:
        print(f"Title: {r['title']}")
        print(f"URL: {r['url']}")
        print(f"Summary: {r['summary']}\n")
else:
    print("Error:", response.status_code)


Title: <span class="qt0">Asthma</span>
URL: None
Summary: What is <span class="qt0">asthma</span>?<p><span class="qt0">Asthma</span> is a chronic (long-term) lung disease. It affects your airways, the tubes that carry air in and out of your lungs. When you have <span class="qt0">asthma</span>, your airways can become inflamed and narrowed. This can cause wheezing, coughing, and tightness in your chest. When these symptoms get worse than usual, it is called an <span class="qt0">asthma</span> attack or flare-up.</p>What causes <span class="qt0">asthma</span>?<p>The exact cause of <span class="qt0">asthma</span> is unknown. Genetics and your environment likely play a role in who gets <span class="qt0">asthma</span>.</p><p>An <span class="qt0">asthma</span> attack can happen when you are exposed to an <span class="qt0">asthma</span> trigger. An <span class="qt0">asthma</span> trigger is something that can set off or worsen your <span class="qt0">asthma</span> symptoms. Different triggers c